In [0]:
%python
df = spark.read.csv("/Volumes/workspace/project/project_files/netflix_titles.csv", header=True, inferSchema=True)

df.display()

In [0]:
%python

# save as managed table
df.write.mode("overwrite").saveAsTable("workspace.project.netflix_titles")

In [0]:
-- checking the data in sql 
SELECT * from workspace.project.netflix_titles
where show_id = 's5023'

In [0]:

--  Checking duplicates 

SELECT show_id, count(*) from workspace.project.netflix_titles
group by show_id
HAVING count(*) > 1

In [0]:
Select * from 
(SELECT upper(title),type, count(*) from workspace.project.netflix_titles
group by upper(title), type
HAVING count(*) > 1
)
-- SELECT * from workspace.project.netflix_titles
-- where title = 'DEATH NOTE'


In [0]:
with base as (SELECT *, 
row_number() OVER (partition by upper(title), type order by show_id) as rn 
FROM workspace.project.netflix_titles
)
Select * from base where rn > 1